## Name:- Dhruvi Mehta ,                          ID:- 202318003
## Name:- Prachi Mehta ,                          ID:- 202318008
## Name:- Simran Dalvi ,                          ID:- 202318042

#### Customer Churn prediction
The goal of this project is to build a customer churn prediction model using user interaction logs with an imaginary music streaming service called Sparkify.

#### Data

Sparkify is imaginary digital music service similar to Spotify. The dataset contains 12GB of user interactions with this fictitious music streaming service.

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

Initialize a spark session

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("sparkify").getOrCreate()
spark

Read the data

In [ ]:
data = spark.read.json("/content/mini_sparkify_event_data.json")

Look at the columns

In [ ]:
data.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



Look at the data, we will do a vertical show

In [ ]:
data.show(vertical=True, n=2)

-RECORD 0-------------------------------
 _corrupt_record | NULL                 
 artist          | Martha Tilston       
 auth            | Logged In            
 firstName       | Colin                
 gender          | M                    
 itemInSession   | 50                   
 lastName        | Freeman              
 length          | 277.89016            
 level           | paid                 
 location        | Bakersfield, CA      
 method          | PUT                  
 page            | NextSong             
 registration    | 1538173362000        
 sessionId       | 29                   
 song            | Rockpools            
 status          | 200                  
 ts              | 1538352117000        
 userAgent       | Mozilla/5.0 (Wind... 
 userId          | 30                   
-RECORD 1-------------------------------
 _corrupt_record | NULL                 
 artist          | Five Iron Frenzy     
 auth            | Logged In            
 firstName      

The most important column in the data set seems to be the `Page` column. It holds all Sparkify pages that the customers have visited.

In [ ]:
data.select("page").dropDuplicates().show(20, False)

+-------------------------+
|page                     |
+-------------------------+
|Cancel                   |
|Submit Downgrade         |
|Thumbs Down              |
|Home                     |
|Downgrade                |
|Roll Advert              |
|Logout                   |
|Save Settings            |
|Cancellation Confirmation|
|About                    |
|Submit Registration      |
|Settings                 |
|Login                    |
|Register                 |
|Add to Playlist          |
|Add Friend               |
|NextSong                 |
|Thumbs Up                |
|Help                     |
|Upgrade                  |
+-------------------------+
only showing top 20 rows



From the list above we can use the `Cancellation Confirmation` to indicate if a particular user churned or not. We are building a model that predicts if a user is going to churn given the history of interactions with the service.

Therfore, it is important to understand the customers that reached the `Cancellation Confirmation` page. These are the customers that churned, and the task is to build a prediction model that can recognize them.

## Data cleaning

Now that we indentified our "goal", we can let go of some of the columns that are not needed for further analysis.

In [ ]:
data = data.drop(*['firstName', 'lastName', 'id_copy'])
data.show(vertical=True, n=2)

-RECORD 0-------------------------------
 _corrupt_record | NULL                 
 artist          | Martha Tilston       
 auth            | Logged In            
 gender          | M                    
 itemInSession   | 50                   
 length          | 277.89016            
 level           | paid                 
 location        | Bakersfield, CA      
 method          | PUT                  
 page            | NextSong             
 registration    | 1538173362000        
 sessionId       | 29                   
 song            | Rockpools            
 status          | 200                  
 ts              | 1538352117000        
 userAgent       | Mozilla/5.0 (Wind... 
 userId          | 30                   
-RECORD 1-------------------------------
 _corrupt_record | NULL                 
 artist          | Five Iron Frenzy     
 auth            | Logged In            
 gender          | M                    
 itemInSession   | 79                   
 length         

Since we are building a model that focuses on a user, remove any null/na values in the userID

In [ ]:
from pyspark.sql.functions import isnan, isnull

In [ ]:
data.filter((isnan(data['userId'])) | (data['userId'].isNull()) | (data['userId'] == "")).count()

8471

In [ ]:
data.filter((data['userId'] == "")).count()

8469

In [ ]:
data = data.filter(data["userId"] != "")
data.filter((isnan(data['userId'])) | (data['userId'].isNull()) | (data['userId'] == "")).count()

0

The times in `registration` and `ts` column are given in milliseconds, we will convert those to seconds. Before that, we will remove any with null values.

In [ ]:
data = data.filter(data['registration'].isNotNull())

In [ ]:
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

In [ ]:
time_unit_udf = F.udf(lambda x: float(x/1000), DoubleType())
data = data.withColumn("registration", time_unit_udf("registration")). \
    withColumn("ts", time_unit_udf("ts"))

data.select('registration', 'ts').show()

+------------+------------------+
|registration|                ts|
+------------+------------------+
| 1538.173362|1538.3521170000001|
|  1538.33163|1538.3521799999999|
| 1538.173362|       1538.352394|
|  1538.33163|       1538.352416|
| 1538.173362|       1538.352676|
|  1538.33163|       1538.352678|
|  1538.33163|       1538.352886|
| 1538.173362|       1538.352899|
| 1538.173362|       1538.352905|
| 1538.173362|       1538.353084|
|  1538.33163|       1538.353146|
|  1538.33163|        1538.35315|
| 1538.173362|1538.3532180000002|
|  1538.33163|       1538.353375|
|  1538.33163|       1538.353376|
| 1538.173362|1538.3534410000002|
|  1538.33163|       1538.353576|
| 1537.365219|       1538.353668|
| 1538.173362|1538.3536869999998|
|  1538.33163|       1538.353744|
+------------+------------------+
only showing top 20 rows



#### Label

As mentioned above we are deriving the label `churn` as a function of the `Page` column. There are only two values to churn, churned or not churned. Therefore this is going to be a boolean column represented by 1 for churned and 0 for not churned.

In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
cancelation_event_udf = F.udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
data = data.withColumn("churn", cancelation_event_udf("page"))
data.filter(data['userId'] == 1749042).describe('churn').show()

+-------+-----+
|summary|churn|
+-------+-----+
|  count|    0|
|   mean| NULL|
| stddev| NULL|
|    min| NULL|
|    max| NULL|
+-------+-----+



The problem here is that a particular user will have multiple entries, since this is a database of user activities log. A user may have visitied number of other pages before reaching the "Cancellation" page. For a user that has churned, we want to make sure that all of his/her logs record 1 for the churn column. For this we will use the Window function from `sql.window`.

For example, user `1749042` above has entries labeled as both churned (1) and not churned (0). We need to make sure that all of his/her entries are labeled as churned. It only takes that one "Cancellation Confirmation" to make a user churn.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as Fsum

In [ ]:
window = Window.partitionBy("userId") \
        .rangeBetween(Window.unboundedPreceding,
                      Window.unboundedFollowing)
data = data.withColumn("churn", Fsum("churn").over(window))


In [ ]:
data.filter(data['userId'] == 1749042).describe('churn').show()

+-------+-----+
|summary|churn|
+-------+-----+
|  count|    0|
|   mean| NULL|
| stddev| NULL|
|    min| NULL|
|    max| NULL|
+-------+-----+



Let's see how many unique users we have

In [ ]:
data.select("userID").dropDuplicates().count()

225

The total churn instances, again a user has multiple entries here

In [ ]:
data.filter(data['churn'] == 1).count()

44864

Now let's create a label dataframe that has one entry per each user

In [ ]:
from pyspark.sql.functions import col

In [ ]:
label = data \
    .select('userId', col('churn').alias('label')) \
    .dropDuplicates()
label.show()

+------+-----+
|userId|label|
+------+-----+
|    10|    0|
|   100|    0|
|100001|    1|
|100002|    0|
|100003|    1|
|100004|    0|
|100005|    1|
|100006|    1|
|100007|    1|
|100008|    0|
|100009|    1|
|100010|    0|
|100011|    1|
|100012|    1|
|100013|    1|
|100014|    1|
|100015|    1|
|100016|    0|
|100017|    1|
|100018|    0|
+------+-----+
only showing top 20 rows



## Feature engineering

Users that have been using the service tend to stay with the service, even as a paying customer, than those that recently signed up.

Every company selling products and services to customers have an idea of a "lifetime value" of a customer. With that in mind we create our first feature.

We converted the times to seconds above. After gaining the lifetime of a user, we convert that to days.

In [ ]:
from pyspark.sql.functions import sum as Fsum, col

In [ ]:
time_since_registration = data \
    .select('userId', 'registration', 'ts') \
    .withColumn('lifetime', (data.ts - data.registration)) \
    .groupBy('userId') \
    .agg({'lifetime': 'max'}) \
    .withColumnRenamed('max(lifetime)', 'lifetime') \
    .select('userId', (col('lifetime') / 3600 / 24).alias('lifetime'))

In [ ]:
time_since_registration.describe('lifetime').show()

+-------+--------------------+
|summary|            lifetime|
+-------+--------------------+
|  count|                 225|
|   mean|7.984568348765434E-5|
| stddev|3.766147001861257E-5|
|    min|3.137268518492596...|
|    max|2.563776736111096...|
+-------+--------------------+



Bussiness big and small not only rely on their customers coming back for more good and services, but also referring the bussiness to their friends and family. It is in our nature to refer things that we like. With this idea in mind we will create another feature.

Again we will user the "Page" column specifically looking for the `Add Friend` page.

In [ ]:
referring_friends = data \
    .select('userID', 'page') \
    .where(data.page == 'Add Friend') \
    .groupBy('userID') \
    .count() \
    .withColumnRenamed('count', 'add_friend')

In [ ]:
referring_friends.describe('add_friend').show()

+-------+------------------+
|summary|        add_friend|
+-------+------------------+
|  count|               206|
|   mean|21.145631067961165|
| stddev| 20.97636661648244|
|    min|                 1|
|    max|               143|
+-------+------------------+



The more songs a user listens to, the more likely they are to enjoy the streaming service and keep their subscription. Thus we add a `total_songs_listened` feature.

In [ ]:
# total songs listened
total_songs_listened = data \
    .select('userID', 'song') \
    .groupBy('userID') \
    .count() \
    .withColumnRenamed('count', 'total_songs')

In [ ]:
total_songs_listened.describe('total_songs').show()

+-------+------------------+
|summary|       total_songs|
+-------+------------------+
|  count|               225|
|   mean|1256.3022222222223|
| stddev|1349.8711205824577|
|    min|                 6|
|    max|              9787|
+-------+------------------+



The more songs a user liked on the streaming service, the more it potentially implies that they enjoy their subscription
and the value they get from it. They are more likely to keep their subscription if they are liking more songs. Thus we create a feature to count the number of songs a user gives a thumbs_up to.

In [ ]:
# thumbs up
thumbs_up = data \
    .select('userID', 'page') \
    .where(data.page == 'Thumbs Up') \
    .groupBy('userID') \
    .count() \
    .withColumnRenamed('count', 'num_thumb_up')

In [ ]:
thumbs_up.describe('num_thumb_up').show()

+-------+-----------------+
|summary|     num_thumb_up|
+-------+-----------------+
|  count|              220|
|   mean|58.00909090909091|
| stddev|66.41518716323678|
|    min|                1|
|    max|              437|
+-------+-----------------+



Similarly, the more songs a user dislikes, the less likely they are to enjoy their subscription and cancel it.

In [ ]:
# thumbs down
thumbs_down = data \
    .select('userID', 'page') \
    .where(data.page == 'Thumbs Down') \
    .groupBy('userID') \
    .count() \
    .withColumnRenamed('count', 'num_thumb_down')

In [ ]:
thumbs_down.describe('num_thumb_down').show()

+-------+------------------+
|summary|    num_thumb_down|
+-------+------------------+
|  count|               203|
|   mean|12.694581280788178|
| stddev| 13.35039288912889|
|    min|                 1|
|    max|                76|
+-------+------------------+



Playlist length counts the number of times a user visited the add to playlist page indicating that they added a song to their playlist. A long playlist implies a user is enjoying several songs and wants frequent access to them. This would likely lead to them keeping their subscription. Thus, we create a feature to compute the lenght of the playlist.

In [ ]:
playlist_length = data.select('userID', 'page') \
    .where(data.page == 'Add to Playlist') \
    .groupby('userID').count() \
    .withColumnRenamed('count', 'playlist_length')

In [ ]:
playlist_length.describe('playlist_length').show()

+-------+------------------+
|summary|   playlist_length|
+-------+------------------+
|  count|               215|
|   mean|30.767441860465116|
| stddev| 33.25210780284287|
|    min|                 1|
|    max|               242|
+-------+------------------+



Avg songs per session allows us to measure how long each user session lasts when they open the application and start a session. The longer the session, the more songs the user is listening too. This could imply that the user is enjoying the application and is less likely to cancel.

In [ ]:
avg_songs_played = data.where('page == "NextSong"') \
    .groupby(['userId', 'sessionId']) \
    .count() \
    .groupby(['userId']) \
    .agg({'count': 'avg'}) \
    .withColumnRenamed('avg(count)', 'avg_songs_played')

In [ ]:
avg_songs_played.describe('avg_songs_played').show()

+-------+------------------+
|summary|  avg_songs_played|
+-------+------------------+
|  count|               225|
|   mean| 71.91858261060032|
| stddev| 44.09146264771613|
|    min|               6.0|
|    max|304.22222222222223|
+-------+------------------+



Artist count measures the amount of different artists the user listens to. A user listening to a wide variety of artists could potentially be enjoying the music application more and is likely to keep their subscription. On the other hand, a user listening to only a handful of artists may not be happy with the current subscription and would be likely to churn.

In [ ]:
artist_count = data \
    .filter(data.page == "NextSong") \
    .select("userId", "artist") \
    .dropDuplicates() \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "artist_count")

In [ ]:
artist_count.describe('artist_count').show()

+-------+-----------------+
|summary|     artist_count|
+-------+-----------------+
|  count|              225|
|   mean|696.3777777777777|
| stddev|603.9518698630803|
|    min|                3|
|    max|             3544|
+-------+-----------------+



Total number of sessions computes how many times the user has started a session on the app. The more sessions implies more visits to the application meaning they are less likely to churn.

In [ ]:
num_sessions = data \
    .select("userId", "sessionId") \
    .dropDuplicates() \
    .groupby("userId") \
    .count() \
    .withColumnRenamed('count', 'num_sessions')

In [ ]:
num_sessions.describe('num_sessions').show()

+-------+------------------+
|summary|      num_sessions|
+-------+------------------+
|  count|               225|
|   mean|14.115555555555556|
| stddev|14.646884657111567|
|    min|                 1|
|    max|               107|
+-------+------------------+



#### Feature table

Now that we have our features, we will join them to create a table of features.

In [ ]:
cols = ["lifetime",
        "total_songs",
        "num_thumb_up",
        'num_thumb_down',
        'add_friend',
        'playlist_length',
        'avg_songs_played',
        'artist_count',
        'num_sessions']
features = [time_since_registration,
            total_songs_listened,
            thumbs_up,
            thumbs_down,
            referring_friends,
            playlist_length,
            avg_songs_played,
            artist_count,
            num_sessions]

In [ ]:
data = features.pop()
while len(features) > 0:
    data = data.join(features.pop(), 'userID', 'outer')

data = data.join(label, 'userID', 'outer').fillna(0)
data.show(vertical=True, n=2)

-RECORD 0--------------------------------
 userId           | 10                   
 num_sessions     | 6                    
 artist_count     | 565                  
 avg_songs_played | 112.16666666666667   
 playlist_length  | 9                    
 add_friend       | 12                   
 num_thumb_down   | 4                    
 num_thumb_up     | 37                   
 total_songs      | 795                  
 lifetime         | 5.176265046296074... 
 label            | 0                    
-RECORD 1--------------------------------
 userId           | 100                  
 num_sessions     | 35                   
 artist_count     | 1705                 
 avg_songs_played | 78.88235294117646    
 playlist_length  | 61                   
 add_friend       | 49                   
 num_thumb_down   | 27                   
 num_thumb_up     | 148                  
 total_songs      | 3214                 
 lifetime         | 6.487377314814825E-5 
 label            | 0             

In [ ]:
data.limit(10).toPandas()

,userId,num_sessions,artist_count,avg_songs_played,playlist_length,add_friend,num_thumb_down,num_thumb_up,total_songs,lifetime,label
0,10,6,565,112.166667,9,12,4,37,795,0.000052,0
1,100,35,1705,78.882353,61,49,27,148,3214,0.000065,0
2,100001,4,125,33.250000,3,2,2,8,187,0.000045,1
3,100002,4,184,48.750000,5,1,0,5,218,0.000160,0
4,100003,2,50,25.500000,2,0,0,3,78,0.000023,1
5,100004,21,733,47.100000,23,19,11,35,1245,0.000172,0
6,100005,5,149,38.500000,3,3,3,7,216,0.000085,1
7,100006,1,26,26.000000,1,4,2,2,44,0.000009,1
8,100007,9,357,47.000000,9,17,6,19,520,0.000115,1
9,100008,6,623,128.666667,30,17,6,37,940,0.000068,0


In [ ]:
data.printSchema()

root
 |-- userId: string (nullable = true)
 |-- num_sessions: long (nullable = true)
 |-- artist_count: long (nullable = true)
 |-- avg_songs_played: double (nullable = false)
 |-- playlist_length: long (nullable = true)
 |-- add_friend: long (nullable = true)
 |-- num_thumb_down: long (nullable = true)
 |-- num_thumb_up: long (nullable = true)
 |-- total_songs: long (nullable = true)
 |-- lifetime: double (nullable = false)
 |-- label: long (nullable = true)



In [ ]:
data.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0|  173|
|    1|   52|
+-----+-----+



### Vector Assembler and Scaling

There are few more steps before we can feed the data into a model. First we will use a vector assempler to create a feature column.

In [ ]:
# Vector assembler
from pyspark.ml.feature import StandardScaler, VectorAssembler
assembler = VectorAssembler(inputCols=cols, outputCol="unScaled_features")
data = assembler.transform(data)

We don't need all of the feature columns before, we will select a few that we need

In [ ]:
data = data.select('userID', 'unScaled_features', 'label')
data.show(vertical=True, n=2)

-RECORD 0---------------------------------
 userID            | 10                   
 unScaled_features | [5.17626504629607... 
 label             | 0                    
-RECORD 1---------------------------------
 userID            | 100                  
 unScaled_features | [6.48737731481482... 
 label             | 0                    
only showing top 2 rows



If you have been paying attention to the min and max values output after our features above, they are everywhere. We need to use a scaler to scale them.

In [ ]:
# scale the features
scaler = StandardScaler(inputCol="unScaled_features", outputCol="features", withStd=True)
scalerModel = scaler.fit(data)
data = scalerModel.transform(data)
data.show(vertical=True, n=2)

-RECORD 0---------------------------------
 userID            | 10                   
 unScaled_features | [5.17626504629607... 
 label             | 0                    
 features          | [1.37441927883800... 
-RECORD 1---------------------------------
 userID            | 100                  
 unScaled_features | [6.48737731481482... 
 label             | 0                    
 features          | [1.72255021155804... 
only showing top 2 rows



### The split

Now that we have our data ready to be transformed and fitted by the models, we split them into training and test set. We will do a 80-20 split

In [ ]:
# train test split
trainTest = data.randomSplit([0.8, 0.2])
trainingDF = trainTest[0]
testDF = trainTest[1]

## Modelling
Four different models were picked:
  * Gradient Boosting Trees,
  * Logistic Regression,
  * Linear SVC,
  * RandomForestClassifier
  
We used Accuracy and f1-score as for evaluating our models. We are really looking for the highest f1 score. Results are shown below.

In [ ]:
from pyspark.ml.classification import GBTClassifier, LogisticRegression, LinearSVC, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
# initialize classifier
GradBoostTree = GBTClassifier()

# Fit the model
cvModel_GradBoostTree = GradBoostTree.fit(trainingDF)

# Make Predictions
results_GradBoostTree = cvModel_GradBoostTree.transform(testDF)
results_GradBoostTree = results_GradBoostTree.select('userID', 'label', 'prediction')
results_GradBoostTree.show(10)

# Get Results
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = evaluator.evaluate(results_GradBoostTree, {evaluator.metricName: "accuracy"})
f1Score = evaluator.evaluate(results_GradBoostTree, {evaluator.metricName: "f1"})
print('Gradient Boosted Trees Metrics:')
print('Accuracy: {:.2f}'.format(accuracy))
print('F1 Score: {:.2f}'.format(f1Score))

+------+-----+----------+
|userID|label|prediction|
+------+-----+----------+
|100004|    0|       0.0|
|100019|    1|       1.0|
|   104|    0|       0.0|
|    11|    0|       0.0|
|   113|    0|       0.0|
|   114|    0|       0.0|
|   123|    0|       0.0|
|   124|    0|       0.0|
|   128|    0|       0.0|
|   131|    0|       0.0|
+------+-----+----------+
only showing top 10 rows

Gradient Boosted Trees Metrics:
Accuracy: 0.86
F1 Score: 0.85


Logistic Regression Classifier

In [ ]:
# initialize classifier
lgr = LogisticRegression()

# Fit the model
cvModel_lgr = lgr.fit(trainingDF)

# Make Predictions
results_lgr = cvModel_lgr.transform(testDF).select('userID', 'label', 'prediction')
results_lgr.show(10)

# Get Results
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = evaluator.evaluate(results_lgr, {evaluator.metricName: "accuracy"})
f1Score = evaluator.evaluate(results_lgr, {evaluator.metricName: "f1"})
print('Logistic Regression Metrics:')
print('Accuracy: {:.2f}'.format(accuracy))
print('F1 Score: {:.2f}'.format(f1Score))

+------+-----+----------+
|userID|label|prediction|
+------+-----+----------+
|100004|    0|       0.0|
|100019|    1|       1.0|
|   104|    0|       0.0|
|    11|    0|       0.0|
|   113|    0|       0.0|
|   114|    0|       0.0|
|   123|    0|       0.0|
|   124|    0|       0.0|
|   128|    0|       0.0|
|   131|    0|       0.0|
+------+-----+----------+
only showing top 10 rows

Logistic Regression Metrics:
Accuracy: 0.92
F1 Score: 0.91


Support Vector Machine

In [ ]:
# initialize classifier
svc = LinearSVC()

# Fit the model
cvModel_svc = svc.fit(trainingDF)

# Make Predictions
results_svc = cvModel_svc.transform(testDF).select('userID', 'label', 'prediction')
results_svc.show(10)

# Get Results
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = evaluator.evaluate(results_svc, {evaluator.metricName: "accuracy"})
f1Score = evaluator.evaluate(results_svc, {evaluator.metricName: "f1"})
print('Support Vector Machine Metrics:')
print('Accuracy: {:.2f}'.format(accuracy))
print('F1 Score: {:.2f}'.format(f1Score))

+------+-----+----------+
|userID|label|prediction|
+------+-----+----------+
|100004|    0|       0.0|
|100019|    1|       0.0|
|   104|    0|       0.0|
|    11|    0|       0.0|
|   113|    0|       0.0|
|   114|    0|       0.0|
|   123|    0|       0.0|
|   124|    0|       0.0|
|   128|    0|       0.0|
|   131|    0|       0.0|
+------+-----+----------+
only showing top 10 rows

Support Vector Machine Metrics:
Accuracy: 0.84
F1 Score: 0.76


Finally, Random Forest Classifier

In [ ]:
rf_classifier = RandomForestClassifier()
cvModel_rf= rf_classifier.fit(trainingDF)

# Make Predictions
results_rf = cvModel_rf.transform(testDF).select('userID', 'label', 'prediction')
results_rf.show(10)

# Get Results
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = evaluator.evaluate(results_rf, {evaluator.metricName: "accuracy"})
f1Score = evaluator.evaluate(results_rf, {evaluator.metricName: "f1"})
print('Random Forest Metrics:')
print('Accuracy: {:.2f}'.format(accuracy))
print('F1 Score: {:.2f}'.format(f1Score))

+------+-----+----------+
|userID|label|prediction|
+------+-----+----------+
|100004|    0|       0.0|
|100019|    1|       1.0|
|   104|    0|       0.0|
|    11|    0|       0.0|
|   113|    0|       0.0|
|   114|    0|       0.0|
|   123|    0|       0.0|
|   124|    0|       0.0|
|   128|    0|       0.0|
|   131|    0|       0.0|
+------+-----+----------+
only showing top 10 rows

Random Forest Metrics:
Accuracy: 0.86
F1 Score: 0.85


From the four models above,  Logistic Regression Classifier
  * Accuracy: 0.92
  * F1 Score: 0.91

## Hybrid model

Ensemble models sometimes produce great results. We shall create a hybrid model to combine our four models above.

Lets combine the results first.

In [ ]:
results_GradBoostTree = results_GradBoostTree.withColumnRenamed("prediction", "prediction_GBT")
results_lgr = results_lgr.select('userId', 'prediction').withColumnRenamed("prediction", "prediction_LGR")
results_svc = results_svc.select('userId', 'prediction').withColumnRenamed("prediction", "prediction_SVC")
results_rf = results_rf.select('userId', 'prediction').withColumnRenamed("prediction", "prediction_RF")
results = results_GradBoostTree.join(results_lgr, 'userID', 'outer')
results = results.join(results_svc, 'userID', 'outer').join(results_rf, 'userID', 'outer')
results.show()

+------+-----+--------------+--------------+--------------+-------------+
|userID|label|prediction_GBT|prediction_LGR|prediction_SVC|prediction_RF|
+------+-----+--------------+--------------+--------------+-------------+
|100004|    0|           0.0|           0.0|           0.0|          0.0|
|100019|    1|           1.0|           1.0|           0.0|          1.0|
|   104|    0|           0.0|           0.0|           0.0|          0.0|
|    11|    0|           0.0|           0.0|           0.0|          0.0|
|   113|    0|           0.0|           0.0|           0.0|          0.0|
|   114|    0|           0.0|           0.0|           0.0|          0.0|
|   123|    0|           0.0|           0.0|           0.0|          0.0|
|   124|    0|           0.0|           0.0|           0.0|          0.0|
|   128|    0|           0.0|           0.0|           0.0|          0.0|
|   131|    0|           0.0|           0.0|           0.0|          0.0|
|   132|    0|           0.0|         

Our hybrid model will simply be an OR function. This means that if a model has labelled if a user will churn, the hybrid model will label that user as churned.

In [ ]:
def hybrid_function(prediction_GBT, prediction_LGR, prediction_SVM, prediction_rf):
    sum_predictions = prediction_GBT + prediction_LGR + prediction_SVM + prediction_rf
    if sum_predictions >= 1:
        return 1.0
    else:
        return 0.0

udf_hybrid_calc_function = F.udf(hybrid_function, DoubleType())
results = results.withColumn("prediction",
                                 udf_hybrid_calc_function(
                                     "prediction_GBT",
                                     "prediction_SVC",
                                     "prediction_LGR",
                                     "prediction_RF"))

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = evaluator.evaluate(results, {evaluator.metricName: "accuracy"})
f1Score = evaluator.evaluate(results, {evaluator.metricName: "f1"})
results.show()
print('Hybrid Metrics:')
print('Accuracy: {:.2f}'.format(accuracy))
print('F1 Score: {:.2f}'.format(f1Score))

+------+-----+--------------+--------------+--------------+-------------+----------+
|userID|label|prediction_GBT|prediction_LGR|prediction_SVC|prediction_RF|prediction|
+------+-----+--------------+--------------+--------------+-------------+----------+
|100004|    0|           0.0|           0.0|           0.0|          0.0|       0.0|
|100019|    1|           1.0|           1.0|           0.0|          1.0|       1.0|
|   104|    0|           0.0|           0.0|           0.0|          0.0|       0.0|
|    11|    0|           0.0|           0.0|           0.0|          0.0|       0.0|
|   113|    0|           0.0|           0.0|           0.0|          0.0|       0.0|
|   114|    0|           0.0|           0.0|           0.0|          0.0|       0.0|
|   123|    0|           0.0|           0.0|           0.0|          0.0|       0.0|
|   124|    0|           0.0|           0.0|           0.0|          0.0|       0.0|
|   128|    0|           0.0|           0.0|           0.0|      

The f1 score and the accuracy stayed the same as the best model.